In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate huggingface_hub librosa soundfile -qq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.5 MB/s eta 0:00:00


In [ ]:
import os
import requests
from IPython.display import Markdown, display
from huggingface_hub import notebook_login
from google.colab import drive
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextStreamer,
    BitsAndBytesConfig,
    pipeline
)
import torch
import librosa

In [ ]:
# Mount Google Drive
drive.mount("/content/drive")

# Configuration
AUDIO_MODEL = "openai/whisper-large-v3"  # Open-source Whisper model
Mistral = "mistralai/Mistral-7B-Instruct-v0.3"  # Open-source Mistral model
audio_filename = "/content/drive/MyDrive/llms/Committee of the Whole  Mar 11, 2025.mp3"

# Authenticate with Hugging Face
notebook_login()

# Load audio file using librosa (resample to 16kHz)
audio_data, sample_rate = librosa.load(audio_filename, sr=16000)

Mounted at /content/drive


In [ ]:
# Create ASR pipeline (only one instance is needed)
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=AUDIO_MODEL,
    device="cuda" if torch.cuda.is_available() else "cpu",
    generate_kwargs={"task": "transcribe", "language": "en"}
)

# Transcribe the audio file in chunks (30s chunks with 5s stride) and extract text
transcription = asr_pipeline(
    {"raw": audio_data, "sampling_rate": sample_rate},
    chunk_length_s=30,
    stride_length_s=5,
    return_timestamps=True
)["text"]

print("Transcription:\n", transcription)

# Prepare LLM inputs with a system message and user prompt
system_message = ("You are an assistant that produces minutes of meetings from transcripts, "
                  "including a summary, key discussion points, takeaways, and action items with owners, in markdown.")
user_prompt = (f"Below is an extract transcript of a council meeting. Please write minutes in markdown, "
               f"including a summary with attendees, location, and date; discussion points; takeaways; "
               f"and action items with owners.\n\n{transcription}")

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription:
  Thank you very much. This is the Arm of Springfield meeting agenda, committee of a whole on March the 11th of 2025. Just a little bit of housekeeping before we start. No hats allowed in chambers, no videos being allowed, or cell phones. Please put my vibrator off and then we'll call the meeting to order. I would like to introduce myself. I'm Mayor Patrick Terrien from the Armour of Springfield. Present is our CEO, Colleen Draper. To my right in the descending order will be the Deputy Mayor, Councillor Ward 1 is Glenn Fuel. Next will be Councillor Ward 2, Andy Kaczynski. Next will be Councillor Ward 3, that's Mark Miller. Next will be Ward 4, Melinda Warm. This point here will go to the approval of the agenda. Can I get a mover and seconder for that, please? Glenn and Melinda. I would like to add under maybe a new business note with regards to King Charles III medal recipients? Be under new business, would that be right? Okay. Anybody else, additions or approvals of the

In [ ]:
# Quantization config for efficient inference (4-bit quantization)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

# Load tokenizer and adjust padding settings
tokenizer = AutoTokenizer.from_pretrained(Mistral)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:

model = AutoModelForCausalLM.from_pretrained(
    Mistral,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16,
    attn_implementation="sdpa"
)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# Generate response using chat template
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer)

outputs = model.generate(
    inputs,
    max_new_tokens=2000,
    streamer=streamer,
    temperature=0.7,
    do_sample=True
)

# Decode and display the final output
final_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
display(Markdown(final_response))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] You are an assistant that produces minutes of meetings from transcripts, including a summary, key discussion points, takeaways, and action items with owners, in markdown.

Below is an extract transcript of a council meeting. Please write minutes in markdown, including a summary with attendees, location, and date; discussion points; takeaways; and action items with owners.

 Thank you very much. This is the Arm of Springfield meeting agenda, committee of a whole on March the 11th of 2025. Just a little bit of housekeeping before we start. No hats allowed in chambers, no videos being allowed, or cell phones. Please put my vibrator off and then we'll call the meeting to order. I would like to introduce myself. I'm Mayor Patrick Terrien from the Armour of Springfield. Present is our CEO, Colleen Draper. To my right in the descending order will be the Deputy Mayor, Councillor Ward 1 is Glenn Fuel. Next will be Councillor Ward 2, Andy Kaczynski. Next will be Councillor Ward 3, that

You are an assistant that produces minutes of meetings from transcripts, including a summary, key discussion points, takeaways, and action items with owners, in markdown.

Below is an extract transcript of a council meeting. Please write minutes in markdown, including a summary with attendees, location, and date; discussion points; takeaways; and action items with owners.

 Thank you very much. This is the Arm of Springfield meeting agenda, committee of a whole on March the 11th of 2025. Just a little bit of housekeeping before we start. No hats allowed in chambers, no videos being allowed, or cell phones. Please put my vibrator off and then we'll call the meeting to order. I would like to introduce myself. I'm Mayor Patrick Terrien from the Armour of Springfield. Present is our CEO, Colleen Draper. To my right in the descending order will be the Deputy Mayor, Councillor Ward 1 is Glenn Fuel. Next will be Councillor Ward 2, Andy Kaczynski. Next will be Councillor Ward 3, that's Mark Miller. Next will be Ward 4, Melinda Warm. This point here will go to the approval of the agenda. Can I get a mover and seconder for that, please? Glenn and Melinda. I would like to add under maybe a new business note with regards to King Charles III medal recipients? Be under new business, would that be right? Okay. Anybody else, additions or approvals of the agenda? With that addition, can I get a mover or a show of hands in support? as unanim and is carried. And then we'll go to the approval of the minutes. That's item number four, mover and seconder, Patrick and Melinda. I know you didn't have your hand up yet, but I just anticipated you were going to do that. So the Committee of the Whole of Minutes, that's February 11th of 2025. We do have an addition down there from Mr. Daniel Page, correction to the minutes there. Would anybody like to comment with regards to item 4, approval of the agenda there? Or I'm sorry, the minutes, sorry. I think the committee of the whole minutes on February 11th are fine. The additions from Mr. Page, I don't think they set the level where we would change it to minutes. Minutes are brief synopsis of the events of whatever date that might be. And they're not meant for a narrative transcription of the entire, in this case here, delegation. Simply said it's too much information there. It's not meant for that. That's what the audio is for. Mark. Thank you, Mayor. Dr. Daniel Page reached out to me with concerns with these draft minutes. And again, he's a member of our community. He's a rate payer. And he took the time and effort to be a delegation in front of council and in fact, in front of the community as a whole. He has suggested to me, and as you can see in his submission to administration and to council, that they don't accurately portray his comments. And I would agree with you in this case, Mayor, that it's not verbatim. I agree with you 100%. But at the same time, the part that is transcribed on the minutes needs to be consistent to the dialogue that the presenter, the delegation provided. And we have Mr. or Dr. Page in the audience today in the chamber, and he's sincerely taking time out of his day once again to share his concerns and to come before us. But what I would like to say is that we need to be accurate in how delegations want to transmit their information, because these are legal instruments that are a record of our deliberations. And if the person who did the delegation feels that they're inadequate or, quite frankly, as he said, this is not consistent to his words and his verbatim, then we need to take attention to that. And I think Councillor Kaczynski, he's in your Thank you. Yes, Dr. Page talked with me about his issues, and he just said that even the headlines of his presentation wasn't included in the minutes. And, you know, it was like misrepresenting and whatever he was saying. So he would like to ask to correct them. And, you know, I don't think so. It's too much to ask for it. And, you know, and we should do subscribe like Mr. Mayor. I know you said that we were doing this for such a long time that why we should change. I guess we live and learn every day. So, you know, if somebody giving us good ideas, we should, you know, embrace that and make sure that we include everybody and we put the proper information. Thank you. Thank you, Councillors Miller and Kaczynski. Anybody else? Go ahead, Councillor Fuel. I just have a question for the CAO, Colleen. Are we following the Municipal Act with the minutes that we're doing at this time? I just believe that we would be doing that. And if we're not, then we can change the minutes to add in. But my understanding is it's not verbatim. And as long as we're following the Act, I'm good with that. Thank you. So I have, I mean, this isn't a new issue. There seems to be a, you know, numerous concerns with inaccurate documentation of delegations. In the minutes, when I spoke with the province, minutes are not a legal document. Under the Act, it says minutes are to be completed without notes or comments and should basically show Council's decisions that are being made. When I talked about delegations, apparently we're going probably above and beyond what other municipalities are doing. We should just be putting the name of the delegate and the title of their presentation. Just last week we were at a conference, I spoke to other municipalities to see how they do their minutes. A lot of them don't do audio or video. We do the audio and post that publicly so anyone can listen to the delegation and hear its entirety. And the suggestion that we should not be trying to summarize delegations. And clearly in our case, it's not working. Thank you. And I guess if I can clarify our CEOs, I think it's working fine. It's just that we're not able to satisfy this individual. and I don't think it rises to the occasion where we would have to reinvent the wheel. It hasn't been an issue before, and this has been going on for quite some time. And there's no misrepresentations at all. There's no intent with regards to that. The feelings are subjective there, and that's what transcript is all about there. So I think at this point here we can vote with regards to the approval of the minutes of a committee of the whole for February the 11th of 2025. Do you have read the ending on that or just i'm just going to show hands of those in support uh that's a counselor warren fuel and terrian those opposed counselors miller and kaczynski is so passed then we'll go to uh That will be item six, and that's the new business there. That's the King Charles III Medal. I just want to bring it to Council's attention there that King Charles III Medals were handed out at a ceremony on March the 4th there at the Steinbach Mennonite Museum by Member of Parliament Ted Folk, MLA Kevin, I think, Gertsen, and MLA Ron Schuller. The recipients are Gary Walker, a retired superintendent of Winnipeg Police, myself, MLA Ron Shuler, and Howie Bradine. So the ceremony was really, really well attended. Many representatives from all the areas within the Ted Fulks area and uh really well uh received there so that's what i want to bring to council's attention um and that's it uh there with regard um go ahead miller constable miller uh thank you mayor um thank you for uh sharing with us the king charles Coronation Awards and congratulations to those who I guess achieved it. What we don't have is the history and I guess the information of how these people earned that. I know there's many in our community who volunteer without recognition, whether it be at the Springfield Food Bank or at many of the churches throughout our community, the community clubs, Helping Hands in Steinbeck, and many others in our community that go unnoticed. And it seems to me that these Coronation Awards, quite frankly, go to political elites or those who are associated with a political affiliation. And you can clearly see that in this. But what I'm saying is I'm going to raise a toast to those people in our community that don't get recognized and definitely deserve recognition because there are just so many that I bump into in our community from the Springfield Agriculture Society to Cook's Creek to Dugald Community Centre. And so, so many that, again, they don't need these recognitions to be, I guess, you know, congratulated on. But please, please, we have to recognize and understand that there are so many in our community that do so much for us. And we couldn't really have a strong community without those people who aren't in those coordination awards. So that's just my comments. And I totally agree with you. Any other comments or questions? Okay, then we can move on to, which one is that one? 6.1, that's unsightly bylaw. Move on to seconder for that. Melinda, Patrick. So this is just for information. Unfortunately, Constable Zillman couldn't make it today. He is the one that kind of oversaw this draft of the unsightly bylaw. So there's a few attachments here. The first is the new draft of the bylaw, the offense schedule proposed. He put together just a document explaining the changes he's proposing. And then just, sorry, today I decided to attach the current bylaw just so you can see the comparison of the two. So it's a lot of information to take in. As I said, Constable Zellman was not able to attend today, but if you've read the document, have any questions, we can do our best to answer them or get some answers from Jesse when he is back at work. Councillor Field? Yeah, I have a question or two. My question is, is when we look at the draft unsightly bylaw explainer from the Springfield Police Service, Down on the bottom, it states that it would be the best interest of the municipality to have two other bylaws, the Darlick Vehicle Bylaw and the Unlawful Dumping Bylaw separately. Are we looking at that too still at this time, or is this just going to be the one for all? Because I noticed in the one that's being presented as the draft has rubbish in there. So I was a little confused on that. Yeah, right now it still is combined as one document. When we did the review, I think those were the points he made is that it probably would serve us better to have separate documents that could take a bit longer to put those documents together. So we would suggest just amending our current bylaw to get it in compliance with our municipal bylaw enforcement bylaw just so that we can start enforcing these things and have a little bit more teeth to it and then continue to work on those additional bylaws and bring them in the future okay thank you for that clarity um counselor miller thank you mayor uh i'm thankful for council or for uh constable zillman for preparing this document i think it's thorough and i think there's uh definitely importance to it One question I have though is under standards and 5.H, I don't really know if I could interpret it properly. So it says regular outdoor burning, the smoke of which causes a nuisance to adjoining properties. Now in my community, particularly because we have such a high level of horse concentration, we do have individuals that do have smudges when the mosquitoes are at high standards. And we don't complain because we have horses as well. We don't actually do a smudge, but it's just a standard kind of thing that people tend to do to protect the horses from, of course, the viruses that mosquitoes carry. We happen to spray our horses periodically and have blankets and other things. But nonetheless, my question here is, what does regular outdoor burning mean? Does it mean garbage? Does it mean smudges? Does it mean other things? And what does the term, if you were in a court of law, I'm not entirely sure both lawyers for and against would argue what regular actually is interpreted as because that's such a less than wholesome kind of a word right it's very interpretive and in law as you know interpretive languages depending on which lawyer you have might fight best but what does constable zillman and or the RM of Springfield interpret as regular burning? Does it mean once a day, once a week, once a month, once every second week? I just think it needs to be more helpful for people, particularly, again, in my ward and I'm sure in others. But again, because they will be affected by, again, if this means smudges or garbage burning, which, of course, garbage burning, I don't absolutely support at all because of the toxins and things, but carcinogens that are emitted. But do you know if there's any clear language on the term regular? So I can't explain regular in this bylaw, but we do have separate burning bylaws that would have more details in it. So we'd probably be regulating the burning under those bylaws anyways, but we do get quite regular complaints about people that are burning, whether they have a burn barrel, and they're not supposed to, we try to regulate that. But if people are just having regular bonfires and it's windy or smoke is constantly blowing into one person's yard, we will get those complaints. So that's something I can take back and we can try and, you know, describe that a little better. Or maybe we just refer, you know, take that out and keep our own burning bylaws as the bylaw that we would enforce those with so um i think this is probably regular nuisance calls that we get and so they're being addressed um but yeah i agree with you that probably has to be. And just for clarification too, for my own point, some constituents or members of the public have said to me that they regularly burn their garbage. Now, again, I'm not entirely sure what that means. Regularly means when the barrel's full. Is garbage burning permitted in our RM at all, at any time, on a private residence? I should know this, but I think the fire chief is online. I'm going to say that burning of garbage is not permitted or shouldn't be. And that's my interpretation, but I want to be entirely accurate on how to transmit that to constituents who ask that question. So I'm not asking to put you on the spot today, but I think it would be helpful for council to know if garbage residential burning is even permitted just for clarity. That's all because it kind of focuses on this too, to some degree, because some people I know are burning garbage every day, all the time, you know, and so that would fit the term regular burning. But if we could just have clarity, that's all I'm asking. I'm not trying to be a nitpicker, but I think this is important for all of us. I was going to say it's not even that clean garbage allowed, but who knows what's in that barrel that they're burning, and then there's noxious smells, and, you know, that's where the problem starts. But I'll definitely clarify that with the fire chief. Councilor Kozinski? And Colleen, you said that the burning garbage, that we have a rule and, you know, an Oak Bank or a Dug, are people complaining more like in the residential area than in the rural area? I think it's more prevalent in the rural area because the towns have garbage pickup. So I don't think it's a problem that they're burning garbage. it's the people that don't want to take additional trips to the transfer station and they're just collecting everything in a barrel and then burning it but that's when you start to get the smells and and you know the smokes that are not fun to have in your backyard so okay thank. Any other questions from council? I see none. Then we can move on to 6.2. That's June district resolutions. Just a reminder for council, same as every year, we kind of have a deadline to submit resolutions to the June district meetings. So June 1st is the deadline. So just a reminder for council to kind of think of any ideas, speak with administration, and we can start to craft those resolutions if there's anything you want to put forward. Okay. Then seeing there's no further questions here i'm prepared prepared to close the meeting to go into in-camera items. Can I get a mover in a second or two to go into a closed meeting, please? Melinda and Glenn? This one here, we're in a closed meeting. Everybody's mics are off thank you we're out of uh closed uh meeting at uh 226 unless there's other issues i'm prepared to adjourn i see none uh can i just get a mover or in a second or two to go to adjourn. I see none. Can I just get a mover and a seconder to go to adjournment? Mark and Melinda. And we'll be closing this meeting of March the 11th of 2025, Committee of a Whole, at 2.27 p.m. Thank you very much. # Meeting Minutes - Committee of a Whole, Arm of Springfield, March 11th, 2025

## Attendees
- Mayor Patrick Terrien
- CEO Colleen Draper
- Deputy Mayor Glenn Fuel (Ward 1)
- Councillor Andy Kaczynski (Ward 2)
- Councillor Mark Miller (Ward 3)
- Councillor Melinda Warm (Ward 4)

## Agenda
- Approval of the agenda
- Approval of the minutes of the Committee of the Whole, February 11th, 2025
- King Charles III Medal recipients
- Unsightly Bylaw (New Business)
- June district resolutions
- In-camera items

## Discussion Points
1. Concerns about the accuracy of delegation transcripts in meeting minutes, with a specific instance raised by Dr. Daniel Page.
2. The need for accurate and consistent documentation of delegations as legal instruments.
3. Debate on the level of detail provided in delegation summaries in meeting minutes.
4. Clarification on the Municipal Act requirements for meeting minutes.
5. Discussion on the purpose and scope of delegation summaries in meeting minutes.
6. Concerns about the focus of the King Charles III Medals on political elites.
7. The need for recognition of unsung community volunteers.
8. Review of the draft unsightly bylaw, including a comparison with the current bylaw.
9. Questions about the interpretation of terms such as "regular outdoor burning" in the unsightly bylaw.
10. Clarification on the permissibility of residential garbage burning in the RM.
11. Deadline for submitting resolutions to the June district meetings.

## Takeaways
- The need for accurate and consistent documentation of delegations in meeting minutes.
- The ongoing concern about the accuracy of transcripts in meeting minutes, particularly with regards to Dr. Daniel Page's comments.
- The need for clearer language and definitions in the unsightly bylaw, particularly with regards to "regular outdoor burning."
- The importance of recognizing and appreciating unsung community volunteers.
- The deadline for submitting resolutions to the June district meetings.

## Action Items
- Review and update the process for documenting delegations in meeting minutes to ensure accuracy and consistency.
- Clarify the terms and definitions used in the unsightly bylaw, particularly "regular outdoor burning."
- Continue recognizing and appreciating unsung community volunteers.
- Submit resolutions to the June district meetings by June 1st, 2025.